# H1 Stock Reconciliation Transformation Solution

## H2 Specifications
Andrew has provide the intended behaviour here:
[Andrews Spec](https://mapofag.sharepoint.com/:w:/s/Project/project_MapofAg_PureFarming/Eb6Rmp1Nk4pBmKmShQm2_RoB4mbEvtc6XRdjKJGynzOESw?e=4UAoe1)


**TimeZone**
To start with, we will need to obtain the timezone of the holding that we are transforming.
For Easy Dairy, this is passed when the holding upload's the file via Provider.
This timezone will then be mapped to determine which hamisphere the holding belongs to. 
We need this to determine the bithing period of an animal:

- **Northern Hamisphere**: 01/01/2025 to 31/12/2025
- **Southern Hamisphere**: 01/07/2025 to 30/06/2026

As for mapping, after discussing with Chris H and looking at which client/hondings are to be using stock rec, we have decided to only map NZ and Australian timezones for now. The rest would be considered as Northern hamisphere.

In [3]:
#Dependencies 
import datetime

In [4]:
#Example of the timezone being fetched 
print('Select a timezone from the list below: Just type in the number')
print('1. Pacific/Auckland')
print('2. Australia/Sydney')
print('3. America/New_York')
tz_choise = input('Enter the number corresponding to your timezone: ')
if tz_choise == '1':
    timezone = 'Pacific/Auckland'
elif tz_choise == '2':
    timezone = 'Australia/Sydney'
elif tz_choise == '3':
    timezone = 'America/New_York'
else:
    print('Invalid choice')
    timezone = tz_choise

print (f'You have selected timezone: {timezone}')

Select a timezone from the list below: Just type in the number
1. Pacific/Auckland
2. Australia/Sydney
3. America/New_York


Enter the number corresponding to your timezone:  1


You have selected timezone: Pacific/Auckland


In [5]:
#Timezone hamisphere mapping

def hamisphere_mapper(timezone):
    if timezone in ['Australia/Sydney', 'Australia/Melbourne', 'Australia/Brisbane', 'Australia/Adelaide', 'Australia/Perth', 'Australia/Hobart']:
        return 'Southern'
    elif timezone in ['Pacific/Auckland']:
        return 'Southern'
    else:
        return 'Northern'  

tz = hamisphere_mapper(timezone)
print(f'The timezone {timezone} is in the {tz} Hemisphere.')

The timezone Pacific/Auckland is in the Southern Hemisphere.
